# MLZoomcamp 2022 - Session #10 - Homework

Author: José Victor

## Building the image

Clone the course repo if you haven't:

ˋˋˋbash
git clone https://github.com/alexeygrigorev/mlbookcamp-code.git
ˋˋˋ

Go to the ˋcourse-zoomcamp/cohorts/2022/05-deployment/homeworkˋ folder and execute the following:

ˋˋˋbash
docker build -t zoomcamp-model:v001 .
ˋˋˋ

Note: If you have troubles building the image, you can use the image we built and published to docker hub: ˋsvizor42/zoomcamp-model:v001ˋ.

In [1]:
!git clone https://github.com/alexeygrigorev/mlbookcamp-code.git

Cloning into 'mlbookcamp-code'...
remote: Enumerating objects: 4071, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 4071 (delta 82), reused 129 (delta 61), pack-reused 3914
Receiving objects: 100% (4071/4071), 9.08 MiB | 7.03 MiB/s, done.
Resolving deltas: 100% (2449/2449), done.


In [2]:
%cd mlbookcamp-code/course-zoomcamp/cohorts/2022/05-deployment/homework

/workspaces/mlzoomcamp2022_jvscursulim/10-kubernetes/homework/mlbookcamp-code/course-zoomcamp/cohorts/2022/05-deployment/homework


In [3]:
!docker build -t zoomcamp-model:v001 .

[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (0/2)                                                         
 => [internal] load build definition from Dockerfile                       0.2s
 => => transferring dockerfile: 302B                                       0.0s
 => [internal] load .dockerignore                                          0.1s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (2/2)                                                         
 => [internal] load build definition from Dockerfile                       0.2s
 => => transferring dockerfile: 302B                                       0.0s
 => [internal] load .dockerignore                                          0.2s
 => => transferring context: 2B                                            0.0s
[+] Building 0.5s (2/3)                                                         
 => [internal] load build definition

In [4]:
!docker images

REPOSITORY       TAG       IMAGE ID       CREATED       SIZE
zoomcamp-model   v001      730ac391bf00   3 days ago    542MB
kindest/node     <none>    de6eb7df13da   2 years ago   1.25GB


## Question 1

Run it to test that it's working locally:

ˋˋˋbash
docker run -it --rm -p 9696:9696 zoomcamp-model:v001
ˋˋˋ

And in another terminal, execute ˋq6_test.pyˋ file:

ˋˋˋbash
python q6_test.py
ˋˋˋ

You should see this:

ˋˋˋ
{'get_card': True, 'get_card_probability': <value>}
ˋˋˋ

Here ˋ<value>ˋ is the probability of getting a credit card. You need to choose the right one.

* ( ) 0.289
* ( ) 0.502
* (X) 0.769
* ( ) 0.972

Now you can stop the container running in Docker.

In [5]:
!python q6_test.py

{'get_card': True, 'get_card_probability': 0.7692649226628628}


## Installing ˋkubectlˋ and ˋkindˋ

You need to install:

* ˋkubectlˋ - [https://kubernetes.io/docs/tasks/tools/](https://kubernetes.io/docs/tasks/tools/) (you might already have it - check before installing)
* ˋkindˋ - [https://kind.sigs.k8s.io/docs/user/quick-start/](https://kind.sigs.k8s.io/docs/user/quick-start/)

https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/

https://www.vivaolinux.com.br/dica/kind-Desenvolvimento-Local-com-Kubernetes

## Question 2

What's the version of ˋkindˋ that you have

Use ˋkind --versionˋ to find out.

### Creating a cluster

Now let's create a cluster with ˋkindˋ:

ˋˋˋ
kind create cluster
ˋˋˋ

And check with ˋkubectlˋ that it was successfully created:

ˋˋˋ
kubectl cluster-info
ˋˋˋ

In [6]:
!kind --version

kind version 0.8.1


In [8]:
!kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.18.2) 🖼
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a nice day! 👋


In [9]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:44885
KubeDNS is running at https://127.0.0.1:44885/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3

What's the smallest deployable computing unit that we can create and manage in Kubernetes (ˋkindˋ in our case)

* ( ) Node
* (X) Pod
* ( ) Deployment
* ( ) Service

## Question 4

Now let's test if everything works. Use ˋkubectlˋ to get the list of running services.

What's the ˋTypeˋ of the service that is already running there

* (X) ClusterIP
* ( ) NodePort
* ( ) LoadBalancer
* ( ) ExternalName

In [10]:
!kubectl get service

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   37s


## Question 5

To be able to use the docker image we previously created (ˋzoomcamp-model:v001ˋ), we need to register it with ˋkindˋ.

What's the command we need to run for that

* ( ) ˋkind create clusterˋ
* ( ) ˋkind build node-imageˋ
* (X) ˋkind load docker-imageˋ
* ( ) ˋkubectl applyˋ

In [1]:
!kind load docker-image zoomcamp-model:v001

Image: "zoomcamp-model:v001" with ID "sha256:730ac391bf000b3b934be8777a0520df72644ed792273d82d180b80d07ae6682" not yet present on node "kind-control-plane", loading...


## Question 6

Now let's create a deployment config (e.g. ˋdeployment.yamlˋ):

ˋˋˋyaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: credit-card
spec:
  selector:
    matchLabels:
      app: credit-card
  replicas: 1
  template:
    metadata:
      labels:
        app: credit-card
    spec:
      containers:
      - name: credit-card
        image: <Image>
        resources:
          requests:
            memory: "64mi"
            cpu: "100m"
          limits:
            memory: <Memory>
            cpu: <CPU>
          ports:
          - containerPort: <Port>
ˋˋˋ

Replace ˋ<Image>ˋ, ˋ<Memory>ˋ, ˋ<CPU>ˋ, ˋ<Port>ˋ with the correct values.

What is the value for ˋ<Port>ˋ

Apply this deployment using the appropriate command and get a list of running Pods. You can see one running Pod.

In [3]:
!kubectl apply -f deployment.yaml

deployment.apps/credit-card created


In [5]:
!kubectl get deployment

NAME          READY   UP-TO-DATE   AVAILABLE   AGE
credit-card   0/1     1            0           85s


In [6]:
!kubectl get pod

NAME                          READY   STATUS    RESTARTS   AGE
credit-card-fbf67ccd4-vqdh6   0/1     Pending   0          2m6s


## Question 7

Let's create a service for this deployment (ˋservice.yamlˋ)ˋ:

ˋˋˋyaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <>
  ports:
  - port: 80
    targetPort: <PORT>
ˋˋˋ

Fill it in. What do we need to write instead of ˋ<>ˋ

Apply this config file.

### Testing the service

We can test our service locally by forwarding the port 9696 on our computer to the port 80 on the service:

ˋˋˋ
kubectl port-forward service/<Service name> 9696:80
ˋˋˋ

Run ˋq6_test.pyˋ (from the homework 5) once again to verify that everything is working. You should get the same result as in Question 1.

### Autoscaling

Now we're going to use a [HorizontalPodAutoscaler]() (HPA for short) that automatically updates a workload resource (such as our deployment), with the aim of automatically scalling the workload to match demand.

Use the following command to create the HPA:

ˋˋˋ
kubectl autoscale deployment credit-card --name credit-card-hpa --cpu-percent=20 --min=1 --max=3
ˋˋˋ

You can check the current status of the new HPA by running:

ˋˋˋ
kubectl get hpa
ˋˋˋ

The output should be similar to the next:

ˋˋˋ
NAME            REFERENCE               TARGETS   MINPODS   MAXPODS   REPLICAS  AGE
credit-card-hpa Deployment/credit-card  1%/20%    1         3         1         27s
ˋˋˋ

ˋTARGETˋ column shows the average CPU consumption across all the Pods controlled by the corresponding deployment. Current CPU consumption is about 0% as there are no clients sending requests to the server.

Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release from the ˋcomponents.yamlˋ manifest:

ˋˋˋ
kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
ˋˋˋ

### Increase the load

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing ˋq6_test.pyˋ script by putting the operator that sends the request to the credit-card service into a loop.

ˋˋˋpython
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
ˋˋˋ

Now you can run this script.

In [8]:
!kubectl apply -f service.yaml

service/credit-card created


In [10]:
!kubectl get svc

NAME          TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)   AGE
credit-card   ClusterIP   10.104.166.224   <none>        80/TCP    8m
kubernetes    ClusterIP   10.96.0.1        <none>        443/TCP   3h45m


In [9]:
!kubectl port-forward service/credit-card 9696:80

error: timed out waiting for the condition


## Question 8 (optional)

Run ˋkubectl get hpa php-apache --watchˋ command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas. What was the maximum amount of the replicas during this test

* ( ) 1
* ( ) 2
* ( ) 3
* ( ) 4

Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled in any way it may happen that the final number of replicas will differ from initial.